In [74]:
import numpy as np

In [75]:
%store -r adult_data_headers
%store -r adult_data_export

In [91]:
%run naive_bayes.ipynb

In [92]:
# %run logistic_regression.ipynb

In [93]:
data = adult_data_export
data_headers = adult_data_headers
# data.shape

In [94]:
def get_new_model(model):
    if model == "LogisticRegression":
        return LogisticRegression(1000)
    else:
        return NaiveBayes()

In [95]:
def evaluate_acc(y,yh):
    tp = np.sum(np.logical_and(yh==1,y==1))
    tn = np.sum(np.logical_and(yh==0,y==0))
    
    acc = (tp+tn)/y.size
    return acc

In [96]:
def get_folds(k, data):
    np.random.shuffle(data)
    
    # Split into k-folds and remove empty splits if not appropriate size
    folds = np.array_split(data, k)
    folds = [x for x in folds if x.size > 0]
    
    return folds

In [97]:
def k_fold_cross_validation(k, data, model_name="LogisticRegression", data_headers=[]):
    np.random.shuffle(data)
    data = data[:5000]
    folds = get_folds(k,data)
    fold_accuracies = []
    for i in range(k):
        print(i)
        model = get_new_model(model_name)
        
        # Get test row, and training set
        test = folds[i]
        train = np.delete(folds, i, 0)
        
        train = np.concatenate(train,axis=0)
        train = np.array(train)
        
        train_X = train[:, :-1]
        train_Y = train[:, -1]
        
        # Fit and predict
        model.fit(train_X, train_Y)
        if(model_name == "NaiveBayes"):
            model.set_headers(data_headers)
            
        test_X = test[:, :-1]
        test_Y = test[:, -1]
        print(test_X)
        model_results = model.predict(test_X)
        
        fold_accuracies.append(evaluate_acc(model_results,test_Y))
    return np.average(fold_accuracies)

In [98]:
k_fold_cross_validation(10,data, model_name="NaiveBayes", data_headers=data_headers)

0
(4500,)
[[    60 235535      9 ...      1      0      0]
 [    36 327435     14 ...      1      0      0]
 [    41  26892     13 ...      1      0      0]
 ...
 [    59 170148     10 ...      1      0      0]
 [    40 222011     14 ...      1      0      0]
 [    23 265148     13 ...      1      0      0]]
1
(4500,)
[[    23 233280     13 ...      1      0      0]
 [    22 315524     10 ...      0      0      0]
 [    25 187653      9 ...      1      0      0]
 ...
 [    24 448026     13 ...      1      0      0]
 [    40  40024     15 ...      1      0      0]
 [    30  55912     13 ...      1      0      0]]
2
(4500,)
[[    40 356934      9 ...      1      0      0]
 [    36 166115     10 ...      1      0      0]
 [    52 201783     10 ...      1      0      0]
 ...
 [    39 158956     10 ...      1      0      0]
 [    22  52262     10 ...      1      0      0]
 [    21 205759      9 ...      1      0      0]]
3
(4500,)
[[    36 185556     13 ...      1      0      0]
 [    24  9

0.8316000000000001

In [ ]:
# # np.random.shuffle(adult_data_export)

# train_X = ionosphere_export[:300, :-1]
# train_y = ionosphere_export[:300, -1]

# test_X = ionosphere_export[300:, :-1]
# test_y = ionosphere_export[300:, -1]

# lr = LogisticRegression(100000)
# lr.fit(train_X, train_y)

# y_pred = lr.predict(test_X)
# lr1_acc = evaluate_acc(y_pred, test_y)


In [ ]:
# lr2 = linear_model.LogisticRegression()
# lr2.fit(train_X, train_y)

# print(lr2.coef_) # returns a matrix of weights (coefficients)'

# y_pred = lr2.predict(test_X)
# lr2_acc = evaluate_acc(y_pred, test_y)

# print("lr1: ", lr1_acc)
# print("lr2: ", lr2_acc)
